# Misestimation from Aggregation: Tutorial and Experiments

This notebook demonstrates the use of the `misestimation_from_aggregation` Python package, which implements the methodology from "Estimating the loss of economic predictability from aggregating firm-level production networks" by Diem et al. (2023).

## Overview

The package provides tools for:
1. **Network Aggregation**: Converting firm-level networks to sector-level representations
2. **Similarity Analysis**: Calculating input/output vector overlaps between firms
3. **Shock Simulation**: Generating synthetic firm-level shocks that maintain sector-level consistency
4. **Economic Impact Assessment**: Comparing firm-level vs sector-level predictive accuracy

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from scipy.sparse import csr_matrix
import warnings
warnings.filterwarnings('ignore')

# Import our package
from misestimation_from_aggregation import (
    NetworkAggregator, 
    SimilarityCalculator, 
    ShockSampler
)

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
np.random.seed(42)

## 1. Example Network from Figure 1

Let's start by recreating the minimal example from Figure 1 of the paper. This network has 11 firms distributed across 5 sectors.

In [ ]:
# Network parameters from Figure 1
n_firms = 11
sector_affiliations = np.array([1, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5])

# Network edges: (supplier, buyer, weight)
edges = [
    (3, 6, 1), (4, 6, 1), (4, 7, 1), (5, 7, 1),
    (6, 1, 1), (6, 2, 1), (7, 10, 1), (7, 11, 1),
    (8, 1, 1), (8, 2, 1), (9, 11, 1)
]

# Create adjacency matrix (firm-level network W)
W = np.zeros((n_firms, n_firms))
for supplier, buyer, weight in edges:
    W[supplier-1, buyer-1] = weight  # Convert to 0-indexed

print(f"Firm-level network W: {W.shape}")
print(f"Number of edges: {np.sum(W > 0)}")
print(f"Total flow: {np.sum(W)}")
print(f"Sectors: {np.unique(sector_affiliations)}")

### Visualize the Networks

In [ ]:
def plot_network(adjacency_matrix, sectors, title, ax):
    """Plot network with sector-based coloring."""
    G = nx.from_numpy_array(adjacency_matrix, create_using=nx.DiGraph)
    
    # Create color map for sectors
    unique_sectors = np.unique(sectors)
    colors = plt.cm.Set3(np.linspace(0, 1, len(unique_sectors)))
    sector_colors = {sector: colors[i] for i, sector in enumerate(unique_sectors)}
    node_colors = [sector_colors[sectors[i]] for i in range(len(sectors))]
    
    # Layout
    pos = nx.spring_layout(G, seed=42)
    
    # Draw network
    nx.draw(G, pos, ax=ax, 
            node_color=node_colors,
            node_size=500,
            arrows=True,
            arrowsize=20,
            edge_color='gray',
            alpha=0.8)
    
    # Add labels
    labels = {i: f'{i+1}\n(S{sectors[i]})' for i in range(len(sectors))}
    nx.draw_networkx_labels(G, pos, labels, ax=ax, font_size=8)
    
    ax.set_title(title)
    return ax

# Plot firm-level and sector-level networks
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Firm-level network
plot_network(W, sector_affiliations, "Firm-level Network (W)", ax1)

# Aggregate to sector level
aggregator = NetworkAggregator()
Z = aggregator.aggregate_to_sectors(W, sector_affiliations)
unique_sectors = np.unique(sector_affiliations)

plot_network(Z, unique_sectors, "Sector-level Network (Z)", ax2)

plt.tight_layout()
plt.show()

print(f"Sector-level network Z: {Z.shape}")
print(f"Total flow preserved: {np.sum(Z)} = {np.sum(W)}")

## 2. Input-Output Vector Analysis

Next, we calculate the overlap coefficients (IOC and OOC) that measure similarity between firms' input and output vectors.

In [ ]:
# Calculate input and output aggregations
supplier_agg = aggregator.aggregate_suppliers(W, sector_affiliations)
buyer_agg = aggregator.aggregate_buyers(W, sector_affiliations)

print("Input aggregation (suppliers → sectors):")
print(f"Volume matrix shape: {supplier_agg['volume'].shape}")
print("Sample input vectors (sectors × firms):")
print(supplier_agg['volume'][:, :5])  # First 5 firms

print("\nOutput aggregation (buyers → sectors):")
print(f"Volume matrix shape: {buyer_agg['volume'].shape}")
print("Sample output vectors (sectors × firms):")
print(buyer_agg['volume'][:, :5])  # First 5 firms

### Calculate Similarity Measures

In [ ]:
# Initialize similarity calculator
similarity_calc = SimilarityCalculator()

# Calculate input vector similarities (IOC)
input_similarities = similarity_calc.calculate_io_similarities(
    W, sector_affiliations, 
    direction="input", 
    measure="overlap_relative"
)

# Calculate output vector similarities (OOC)
output_similarities = similarity_calc.calculate_io_similarities(
    W, sector_affiliations, 
    direction="output", 
    measure="overlap_relative"
)

print("Input vector overlaps by sector:")
for sector, sim_matrix in input_similarities.items():
    if sim_matrix.size > 0:
        print(f"{sector}: {sim_matrix.shape}")
        if sim_matrix.size > 1:
            print(f"  Sample similarities: {sim_matrix.flatten()[:5]}")

print("\nOutput vector overlaps by sector:")
for sector, sim_matrix in output_similarities.items():
    if sim_matrix.size > 0:
        print(f"{sector}: {sim_matrix.shape}")
        if sim_matrix.size > 1:
            print(f"  Sample similarities: {sim_matrix.flatten()[:5]}")

### Examine Specific Results from the Paper

The paper mentions specific examples:
- Sector 3 firms have input overlap of 1 (both buy from sector 2)
- Sector 5 firms have input overlap of 0.5 (firm 10 buys only from sector 3, firm 11 buys from sectors 3 and 4)
- Sector 3 firms have output overlap of 0 (firm 6 sells only to sector 1, firm 7 only to sector 5)

In [ ]:
# Examine specific cases mentioned in the paper
print("Detailed analysis of specific sectors:")

# Sector 3 analysis (firms 6 and 7, i.e., indices 5 and 6)
sector_3_firms = np.where(sector_affiliations == 3)[0]
print(f"\nSector 3 firms: {sector_3_firms + 1} (0-indexed: {sector_3_firms})")

# Input vectors for sector 3 firms
print("Input vectors (what they buy from each sector):")
for i, firm_idx in enumerate(sector_3_firms):
    firm_inputs = supplier_agg['volume'][:, firm_idx]
    print(f"  Firm {firm_idx + 1}: {firm_inputs}")

# Output vectors for sector 3 firms  
print("Output vectors (what they sell to each sector):")
for i, firm_idx in enumerate(sector_3_firms):
    firm_outputs = buyer_agg['volume'][:, firm_idx]
    print(f"  Firm {firm_idx + 1}: {firm_outputs}")

# Sector 5 analysis (firms 10 and 11, i.e., indices 9 and 10)
sector_5_firms = np.where(sector_affiliations == 5)[0]
print(f"\nSector 5 firms: {sector_5_firms + 1} (0-indexed: {sector_5_firms})")

print("Input vectors (what they buy from each sector):")
for i, firm_idx in enumerate(sector_5_firms):
    firm_inputs = supplier_agg['volume'][:, firm_idx]
    print(f"  Firm {firm_idx + 1}: {firm_inputs}")

## 3. Synthetic Shock Generation

Now we demonstrate the core contribution of the paper: generating synthetic firm-level shocks that maintain sector-level consistency.

In [ ]:
# Create empirical shock: 100% shock to firm 3 (index 2), 0% to all others
empirical_shock = np.zeros(n_firms)
empirical_shock[2] = 1.0  # Firm 3 gets 100% shock

print(f"Empirical shock vector: {empirical_shock}")
print(f"Shocked firm: {np.where(empirical_shock > 0)[0] + 1} (sector {sector_affiliations[2]})")

# Initialize shock sampler
sampler = ShockSampler(random_seed=100)  # For reproducibility

# Generate synthetic shocks
synthetic_shocks = sampler.sample_firm_level_shocks(
    firm_shock=empirical_shock,
    network=W,
    sector_affiliations=sector_affiliations,
    n_scenarios=10,
    sample_mode="empirical",
    tracker=False,
    silent=True
)

print(f"\nGenerated synthetic shocks: {synthetic_shocks.shape}")
print(f"Number of scenarios: {synthetic_shocks.shape[1]}")

### Analyze Synthetic Shocks

In [ ]:
# Analyze the synthetic shocks
print("Synthetic shock analysis:")
print(f"Non-zero elements per scenario: {np.sum(synthetic_shocks > 0, axis=0)}")

# Check which firms get shocked in each scenario
for scenario in range(min(5, synthetic_shocks.shape[1])):
    shocked_firms = np.where(synthetic_shocks[:, scenario] > 0)[0]
    shocked_sectors = sector_affiliations[shocked_firms]
    print(f"Scenario {scenario + 1}: Firms {shocked_firms + 1} in sectors {shocked_sectors}")

# Verify sector-level consistency
s_in = np.sum(W, axis=0)   # In-strength
s_out = np.sum(W, axis=1)  # Out-strength

print("\nSector-level shock consistency check:")
sector_2_mask = sector_affiliations == 2

# Original sector 2 shocks
orig_in_shock = np.sum(s_in[sector_2_mask] * empirical_shock[sector_2_mask])
orig_out_shock = np.sum(s_out[sector_2_mask] * empirical_shock[sector_2_mask])
print(f"Original sector 2 in-shock: {orig_in_shock:.3f}")
print(f"Original sector 2 out-shock: {orig_out_shock:.3f}")

# Synthetic sector 2 shocks
print("\nSynthetic sector 2 shocks:")
for scenario in range(min(5, synthetic_shocks.shape[1])):
    synth_shock = synthetic_shocks[:, scenario]
    synth_in_shock = np.sum(s_in[sector_2_mask] * synth_shock[sector_2_mask])
    synth_out_shock = np.sum(s_out[sector_2_mask] * synth_shock[sector_2_mask])
    print(f"Scenario {scenario + 1}: in={synth_in_shock:.3f}, out={synth_out_shock:.3f}")

### Visualize Shock Distributions

In [ ]:
# Visualize shock distributions
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))

# 1. Heatmap of all synthetic shocks
im1 = ax1.imshow(synthetic_shocks, aspect='auto', cmap='Reds')
ax1.set_xlabel('Scenario')
ax1.set_ylabel('Firm')
ax1.set_title('Synthetic Shock Matrix')
plt.colorbar(im1, ax=ax1)

# 2. Distribution of shock values
shock_values = synthetic_shocks[synthetic_shocks > 0]
ax2.hist(shock_values, bins=20, alpha=0.7, color='red')
ax2.set_xlabel('Shock Intensity')
ax2.set_ylabel('Frequency')
ax2.set_title('Distribution of Non-Zero Shock Values')

# 3. Number of shocked firms per scenario
shocked_counts = np.sum(synthetic_shocks > 0, axis=0)
ax3.bar(range(1, len(shocked_counts) + 1), shocked_counts, color='blue', alpha=0.7)
ax3.set_xlabel('Scenario')
ax3.set_ylabel('Number of Shocked Firms')
ax3.set_title('Shocked Firms per Scenario')

# 4. Sector-level shock consistency
sector_in_shocks = []
sector_out_shocks = []
for scenario in range(synthetic_shocks.shape[1]):
    synth_shock = synthetic_shocks[:, scenario]
    in_shock = np.sum(s_in[sector_2_mask] * synth_shock[sector_2_mask])
    out_shock = np.sum(s_out[sector_2_mask] * synth_shock[sector_2_mask])
    sector_in_shocks.append(in_shock)
    sector_out_shocks.append(out_shock)

scenarios = range(1, len(sector_in_shocks) + 1)
ax4.plot(scenarios, sector_in_shocks, 'o-', label='In-strength shock', color='blue')
ax4.plot(scenarios, sector_out_shocks, 's-', label='Out-strength shock', color='red')
ax4.axhline(orig_in_shock, color='blue', linestyle='--', alpha=0.7, label='Original in-shock')
ax4.axhline(orig_out_shock, color='red', linestyle='--', alpha=0.7, label='Original out-shock')
ax4.set_xlabel('Scenario')
ax4.set_ylabel('Sector-level Shock')
ax4.set_title('Sector-level Shock Consistency')
ax4.legend()

plt.tight_layout()
plt.show()

## 4. Economic Impact Analysis

Now let's analyze the economic impact using influence vectors based on PageRank, as mentioned in the paper.

In [ ]:
# Calculate influence vectors using PageRank
alpha = 0.5  # Labor share (dampening factor)

# Firm-level influence vector
G_firm = nx.from_numpy_array(W.T, create_using=nx.DiGraph)  # Transpose for out-strength normalization
firm_influence = nx.pagerank(G_firm, alpha=1-alpha)
firm_influence_vec = np.array([firm_influence[i] for i in range(n_firms)])

# Sector-level influence vector  
G_sector = nx.from_numpy_array(Z.T, create_using=nx.DiGraph)
sector_influence = nx.pagerank(G_sector, alpha=1-alpha)
sector_influence_vec = np.array([sector_influence[i] for i in range(len(np.unique(sector_affiliations)))])

print(f"Firm-level influence vector: {firm_influence_vec}")
print(f"Sector-level influence vector: {sector_influence_vec}")

In [ ]:
# Calculate economy-wide losses
firm_level_losses = []
for scenario in range(synthetic_shocks.shape[1]):
    loss = np.dot(firm_influence_vec, synthetic_shocks[:, scenario])
    firm_level_losses.append(loss)

# Calculate sector-level equivalent
# First, calculate initial sector shocks from firm-level data
sector_initial_shocks = np.zeros(len(np.unique(sector_affiliations)))
for i, sector in enumerate(np.unique(sector_affiliations)):
    sector_mask = sector_affiliations == sector
    sector_initial_shocks[i] = np.sum(empirical_shock[sector_mask] * s_out[sector_mask]) / max(np.sum(s_out[sector_mask]), 1e-10)

sector_level_loss = np.dot(sector_influence_vec, sector_initial_shocks)

print(f"Economy-wide losses from firm-level analysis: {firm_level_losses}")
print(f"Economy-wide loss from sector-level analysis: {sector_level_loss}")
print(f"Mean firm-level loss: {np.mean(firm_level_losses):.6f}")
print(f"Standard deviation: {np.std(firm_level_losses):.6f}")

### Visualize Economic Impact Comparison

In [ ]:
# Create visualization comparing firm-level vs sector-level predictions
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 1. Distribution of firm-level losses
ax1.hist(firm_level_losses, bins=15, alpha=0.7, color='skyblue', 
         label='Firm-level predictions', density=True)
ax1.axvline(sector_level_loss, color='red', linestyle='--', linewidth=2,
           label='Sector-level prediction')
ax1.axvline(np.mean(firm_level_losses), color='blue', linestyle='-', linewidth=2,
           label='Mean firm-level')
ax1.set_xlabel('Economy-wide Output Loss')
ax1.set_ylabel('Density')
ax1.set_title('Economic Impact: Firm-level vs Sector-level Predictions')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Scenario-by-scenario comparison
scenarios = range(1, len(firm_level_losses) + 1)
ax2.plot(scenarios, firm_level_losses, 'o-', color='blue', 
         label='Firm-level losses', alpha=0.8)
ax2.axhline(sector_level_loss, color='red', linestyle='--', linewidth=2,
           label='Sector-level loss')
ax2.fill_between(scenarios, 
                 [sector_level_loss] * len(scenarios),
                 firm_level_losses, 
                 alpha=0.2, color='gray',
                 label='Prediction error')
ax2.set_xlabel('Scenario')
ax2.set_ylabel('Economy-wide Output Loss')
ax2.set_title('Loss Predictions by Scenario')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate prediction error statistics
prediction_errors = np.array(firm_level_losses) - sector_level_loss
print(f"\nPrediction Error Analysis:")
print(f"Mean error: {np.mean(prediction_errors):.6f}")
print(f"Standard deviation of errors: {np.std(prediction_errors):.6f}")
print(f"Range of errors: [{np.min(prediction_errors):.6f}, {np.max(prediction_errors):.6f}]")
print(f"Relative error (% of sector prediction): {100 * np.std(prediction_errors) / abs(sector_level_loss):.2f}%")

## 5. Sensitivity Analysis

Let's explore how the results change with different parameters and network structures.

In [ ]:
# Sensitivity analysis: different shock magnitudes
shock_magnitudes = [0.25, 0.5, 0.75, 1.0]
results_by_magnitude = {}

for magnitude in shock_magnitudes:
    # Create shock
    test_shock = np.zeros(n_firms)
    test_shock[2] = magnitude  # Firm 3
    
    # Generate synthetic shocks
    synth_shocks = sampler.sample_firm_level_shocks(
        firm_shock=test_shock,
        network=W,
        sector_affiliations=sector_affiliations,
        n_scenarios=20,
        silent=True
    )
    
    # Calculate losses
    losses = [np.dot(firm_influence_vec, synth_shocks[:, s]) for s in range(synth_shocks.shape[1])]
    
    results_by_magnitude[magnitude] = {
        'losses': losses,
        'mean': np.mean(losses),
        'std': np.std(losses)
    }

# Plot sensitivity results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Mean losses vs shock magnitude
magnitudes = list(results_by_magnitude.keys())
means = [results_by_magnitude[m]['mean'] for m in magnitudes]
stds = [results_by_magnitude[m]['std'] for m in magnitudes]

ax1.errorbar(magnitudes, means, yerr=stds, marker='o', capsize=5, capthick=2)
ax1.set_xlabel('Shock Magnitude')
ax1.set_ylabel('Mean Economy-wide Loss')
ax1.set_title('Loss vs Shock Magnitude')
ax1.grid(True, alpha=0.3)

# Standard deviation vs shock magnitude
ax2.plot(magnitudes, stds, 'o-', color='red')
ax2.set_xlabel('Shock Magnitude')
ax2.set_ylabel('Standard Deviation of Losses')
ax2.set_title('Prediction Uncertainty vs Shock Magnitude')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Sensitivity Analysis Results:")
for magnitude in magnitudes:
    result = results_by_magnitude[magnitude]
    print(f"Magnitude {magnitude}: Mean={result['mean']:.6f}, Std={result['std']:.6f}")

## 6. Comparison with Different Similarity Measures

Let's compare different similarity measures to understand their behavior.

In [ ]:
# Compare different similarity measures
similarity_measures = ['jaccard', 'overlap_relative', 'cosine']
similarity_results = {}

for measure in similarity_measures:
    input_sim = similarity_calc.calculate_io_similarities(
        W, sector_affiliations, 
        direction="input", 
        measure=measure
    )
    
    output_sim = similarity_calc.calculate_io_similarities(
        W, sector_affiliations, 
        direction="output", 
        measure=measure
    )
    
    similarity_results[measure] = {
        'input': input_sim,
        'output': output_sim
    }

# Create comparison table
print("Similarity Measure Comparison:")
print("=" * 60)

for measure in similarity_measures:
    print(f"\n{measure.upper()}:")
    
    input_sim = similarity_results[measure]['input']
    output_sim = similarity_results[measure]['output']
    
    for sector_name in input_sim.keys():
        if input_sim[sector_name].size > 1:
            in_vals = input_sim[sector_name][np.tril_indices_from(input_sim[sector_name], k=-1)]
            out_vals = output_sim[sector_name][np.tril_indices_from(output_sim[sector_name], k=-1)]
            
            if len(in_vals) > 0:
                print(f"  {sector_name}: Input={in_vals[0]:.3f}, Output={out_vals[0]:.3f}")

## 7. Conclusions and Key Insights

This notebook has demonstrated the key concepts from the "Misestimation from Aggregation" paper:

### Key Findings:

1. **Network Aggregation Effects**: When we aggregate firm-level networks to sector-level, we lose important heterogeneity information.

2. **Similarity Measures**: Different firms within the same sector can have very different input/output patterns, leading to varying overlap coefficients.

3. **Shock Propagation**: The synthetic shock generation algorithm successfully creates heterogeneous firm-level shocks that maintain sector-level consistency.

4. **Economic Prediction Errors**: Sector-level analysis can lead to systematic misestimation of economic impacts compared to firm-level analysis.

### Practical Implications:

- **Policy Making**: Sector-level economic models may underestimate or overestimate the true impact of economic shocks.
- **Risk Assessment**: Firm-level heterogeneity creates additional uncertainty that should be considered in economic forecasting.
- **Network Analysis**: The methodology provides tools for better understanding production network structure and shock propagation.

### Next Steps:

- Apply the methodology to larger, real-world production networks
- Explore different shock scenarios and network structures
- Investigate the relationship between network topology and prediction accuracy
- Develop improved aggregation methods that preserve more firm-level information

In [ ]:
# Summary statistics
print("SUMMARY STATISTICS")
print("=" * 50)
print(f"Network size: {n_firms} firms, {len(np.unique(sector_affiliations))} sectors")
print(f"Network density: {np.sum(W > 0) / (n_firms * n_firms):.3f}")
print(f"Total network flow: {np.sum(W)}")
print(f"")
print(f"Synthetic shocks generated: {synthetic_shocks.shape[1]} scenarios")
print(f"Average shocked firms per scenario: {np.mean(np.sum(synthetic_shocks > 0, axis=0)):.1f}")
print(f"")
print(f"Economy-wide loss predictions:")
print(f"  Firm-level mean: {np.mean(firm_level_losses):.6f}")
print(f"  Firm-level std:  {np.std(firm_level_losses):.6f}")
print(f"  Sector-level:    {sector_level_loss:.6f}")
print(f"  Prediction error: {100 * abs(np.mean(firm_level_losses) - sector_level_loss) / abs(sector_level_loss):.2f}%")